In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf
from ibapi.client import *
from ibapi.wrapper import *
from ibapi.contract import *
from ibapi.order import *
import ibapi

import requests
import bs4 as bs
import datetime
import time
import threading
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV  
import joblib

import talib as ta
from talib import MA_Type

In [2]:
init_event = threading.Event() # init event
id_event = threading.Event()
mkt_event = threading.Event()
hist_event = threading.Event()
order_event = threading.Event()
evec_event = threading.Event()
port_event = threading.Event()
value_event = threading.Event()

In [3]:
# define the App class
class App(EClient, EWrapper):
    def __init__(self, address, port, cid):
        EClient.__init__(self, self)
        # # list to store data
        self.bar_dict = {}
        self.mkt_price = []
        # create a connection with IBKR
        self.connect(address, port, cid)
        self.last_portfolio = pd.DataFrame(columns=["ticker","quantity","marketPrice","marketValue","averageCost","unrealizedPNL","realizedPNL"])
        self.value = 0
        # start client
        thread = threading.Thread(target=self.run)
        thread.start()
        init_event.set()


    def nextValidId(self, orderId: int):
        # provide a new order id for each of my requests
        super().nextValidId(orderId)
        logging.debug("setting nextValidOrderId: %d", orderId)
        self.nextValidOrderId = orderId
        print("NextValidId:", orderId)
        id_event.set()

    def tickPrice(self, reqId: int, tickType: int, price: float, attrib: ibapi.common.TickAttrib):
            print("Tick Price. Ticker Id:", reqId, "tickType:", tickType, "Price:", price)
            if tickType == 9: # if tickType is Close Price
                self.mkt_price.append([reqId, price])
                mkt_event.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.bar_dict.keys():
            self.bar_dict[reqId] = []
        self.bar_dict[reqId].append(vars(bar))
        
    def historicalDataEnd(self, reqId, start, end):
        print(f"end of historicalData")
        hist_event.set()


    # implement code to monitor trade status and receive confirmation of the trade
    def openOrder(self, orderId: OrderId, contract: Contract, order: Order, orderstate: OrderState):
        # openorder callback
        print(f"openOrder. orderId:{orderId}, contract:{contract}, order:{order}")
        order_event.set()

    def orderStatus(self, orderId: OrderId, status: str, filled: float, reamining: float, avgFillPrice: float,
                    permId: int, parenId: int, lastFillPrice: float, clientId: int, whyHeld:str, mktCapPrice: float):
                    # orderstatus callback
        print(f"orderStatus. orderId: {orderId}, status: {status}, filled: {filled}, remaining:{reamining}, avgFillPrice: {avgFillPrice}, permId:{permId}, parentId:{parenId}, lastFillPrice: {lastFillPrice}, clientId: {clientId}, whyHeld: {whyHeld}, mktCapPrice:{mktCapPrice}")

    def execDetails(self, reqId: int, contract: Contract, execution: Execution):
        print(f"execDetails. reqId: {reqId}, contract: {contract}, execution: {execution}")
        evec_event.set()
    # basically a summary

    def commissionReport(self, commissionReport: CommissionReport):
        super().commissionReport(commissionReport)
        print("CommissionReport.", commissionReport)
        
    # called when query portfolio information
    def updatePortfolio(self, contract, position, marketPrice, marketValue, averageCost, unrealizedPNL, realizedPNL, accountName):
        self.last_portfolio = pd.concat([self.last_portfolio,
                                        pd.DataFrame([[contract.symbol, position, marketPrice, marketValue, averageCost, unrealizedPNL, realizedPNL]],
                                                    columns=self.last_portfolio.columns)],
                                        ignore_index=True)
        port_event.set()

    # query total value
    def accountSummary(self, reqId: int, account: str, tag: str, value: str, currency: str):
        self.value = float(value)
        print("Total value of the account: ", self.value)
    
    def accountSummaryEnd(self, reqId: int):
        print('end of account summary')
        value_event.set()

In [4]:
# Connect to the TWS API
app = App('127.0.0.1', 7497, 1000)
init_event.wait() # wait until it's connected
init_event.clear()

NextValidId: 1


In [5]:
app.reqMarketDataType(1) # in case it doesn't work, change 1 to 3

ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR:ibapi.wrapper:ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR:ibapi.wrapper:ERROR -1 2158 Sec-def data farm connection is OK:secdefil


In [6]:
#get S&P tickers list from wikipedia
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

tickers = []

for row in table.findAll('tr')[1:]:
    ticker_name = row.findAll('td')[0].text
    tickers.append(ticker_name)

#clean list
tickers = [s.replace('\n', '') for s in tickers]

#get data from yfin
start = datetime.datetime(2010, 1, 1)
end = datetime.datetime.now().strftime("%Y-%m-%d")
data = yf.download(tickers, start=start, end=end)

print(data)
#TODO: note we might also need to get data from tws.

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BRK.B: No timezone found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
                            Adj Close                                     \
                                    A        AAL         AAP        AAPL   
Date                                                                       
2010-01-04 00:00:00-05:00   20.301353   4.496878   37.163273    6.515213   
2010-01-05 00:00:00-05:00   20.080828   5.005957   36.942390    6.526476   
2010-01-06 00:00:00-05:00   20.009489   4.798553   37.264496    6.422665   
2010-01-07 00:00:00-05:00   19.983553   4.939964   37.255287    6.410791   
2010-01-08 00:00:00-05:00   19.977057   4.845691   37.402557    6.453412   
...                               ...        ...         ...         ...   
2023-01-26 00:00:00-05:00  156.229996  16.610001  147.619995  143.960007   
2023-01-27 00:00:00-05:00  15

In [7]:
data.to_pickle('SP500_data.pkl')

# data cleaning part

In [8]:
#clean data
df = data.stack().reset_index().rename(index=str, columns={"level_1": "Symbol"}).sort_values(['Symbol','Date'])
df.set_index('Date', inplace=True)

#drop tickers not in the list
tickers.remove("BRK.B")
tickers.remove("BF.B")
tickers.remove("FTV")
tickers.remove("GEHC")

#sort by group
groups = df.groupby("Symbol")
Ticker_Data = {}

for t in tickers:
    Ticker_Data[t] = groups.get_group(t)

In [9]:
df

,Symbol,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2010-01-04 00:00:00-05:00,A,20.301353,22.389128,22.625179,22.267525,22.453505,3815561.0
2010-01-05 00:00:00-05:00,A,20.080828,22.145924,22.331903,22.002861,22.324751,4186031.0
2010-01-06 00:00:00-05:00,A,20.009489,22.067240,22.174536,22.002861,22.067240,3243779.0
2010-01-07 00:00:00-05:00,A,19.983553,22.038628,22.045780,21.816881,22.017166,3095172.0
2010-01-08 00:00:00-05:00,A,19.977057,22.031473,22.067240,21.745352,21.917025,3733918.0
...,...,...,...,...,...,...,...
2023-01-26 00:00:00-05:00,ZTS,168.240005,168.240005,168.300003,166.119995,166.869995,1530700.0
2023-01-27 00:00:00-05:00,ZTS,165.179993,165.179993,167.690002,164.500000,166.979996,1831700.0
2023-01-30 00:00:00-05:00,ZTS,164.699997,164.699997,165.630005,163.820007,164.000000,1950600.0


In [10]:
last_X = [] # use the last day's data to make prediction

for t in tickers:
    last_X.append(Ticker_Data[t].iloc[-1,:].drop(['Symbol'],1))

    Ticker_Data[t]['High Shifted']=Ticker_Data[t]['High'].shift(1)
    Ticker_Data[t]['Low Shifted'] = Ticker_Data[t]['Low'].shift(1)
    Ticker_Data[t]['Close Shifted'] = Ticker_Data[t]['Close'].shift(1)
    
    Ticker_Data[t]['Upper BBand'], Ticker_Data[t]['Middle BBand'],Ticker_Data[t]['Lower BBand']= ta.BBANDS(Ticker_Data[t]['Close Shifted'],
                                                                                                       timeperiod=20)

    Ticker_Data[t]['Macd'], Ticker_Data[t]['Macd Signal'],Ticker_Data[t]['Macd Hist'] = ta.MACD(Ticker_Data[t]['Close Shifted'], fastperiod=12, slowperiod=26, 
                                                               signalperiod=9)
    Ticker_Data[t]['Momentum'] = ta.MOM(Ticker_Data[t]['Close Shifted'],timeperiod=10)
    
    Ticker_Data[t]['RSI'] = ta.RSI(np.array(Ticker_Data[t]['Close Shifted']), timeperiod=12)

    Ticker_Data[t]['ROC'] = ta.ROC(np.array(Ticker_Data[t]['Close Shifted']), timeperiod=10)

    Ticker_Data[t]['Returns'] = np.log(Ticker_Data[t]['Open'].shift(-7)/Ticker_Data[t]['Open']) # I want to predict the return of 7 days after today.

    Ticker_Data[t].dropna(inplace =True)

    # get rid of the 30% data in the middle to reduce noise.
    Ticker_Data[t]=Ticker_Data[t].sort(columns='Returns')
    Ticker_Data[t]=pd.concat([Ticker_Data[t].iloc[:np.floor(len(Ticker_Data[t]['Returns'])*0.35),:],
                            Ticker_Data[t].iloc[np.floor(len(Ticker_Data[t]['Returns'])*0.85):,:]])

last_X = pd.concat(last_X).to_numpy()

/tmp/ipykernel_23883/96338543.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ticker_Data[t]['High Shifted']=Ticker_Data[t]['High'].shift(1)
/tmp/ipykernel_23883/96338543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ticker_Data[t]['Low Shifted'] = Ticker_Data[t]['Low'].shift(1)
/tmp/ipykernel_23883/96338543.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

# stock selection

In [11]:
X = []
Y = []
for i in tickers:
    X.append(Ticker_Data[i].drop(['Symbol','Returns', 'High Shifted', 'Low Shifted', 'Close Shifted'],1))
    Y.append(Ticker_Data[i]['Returns'].apply(lambda x:1 if (x>np.mean(list(Ticker_Data[i]['Returns']) & (x>0))) else -1)) # 1 or -1
# turn list into dataframe, then into np.array
X = pd.concat(X).to_numpy()
Y = pd.concat(Y).to_numpy()

In [ ]:
# SVM pipeline
# only need to excute it once if it takes too long
# TODO: comment out this cell if it takes too long
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', SVC())
])
parameters = {'kernel':['rbf'],'C':[0.01,0.03,0.1,0.3,1,3,10],\
              'gamma':[1e-4,3e-4,1e-3,3e-3,0.01,0.03,0.1,0.3,1]}
# gridsearch
clf = GridSearchCV(pipe,parameters,scoring=['roc_auc'],cv=10)
clf.fit(X, Y)

In [ ]:
print(clf.cv_results_)
print(clf.grid_scores_)
print(clf.best_score_)
print(clf.best_params_)

# save to model to a file
joblib.dump(clf, 'svm_model.joblib')

In [ ]:
clf = joblib.load('svm_model.joblib')
# retrain the model with updated data
clf.fit(X, Y)

# select top ten stocks

In [ ]:
probs = clf.predict_proba(last_X)[:,1] #  how close it is to 1

# sort probabilities in descending order
sorted_probs = np.sort(probs)[::-1]

# set threshold for top ten closest to 1
threshold = sorted_probs[9]

# set top ten closest to 1 to 1, and the rest to 0
tmp_index = np.zeros(len(probs))
tmp_index[probs >= threshold] = 1

portfolio_list = tickers[tmp_index]
portfolio_list

# experiment

In [1]:
# import yfinance as yf

# # Get stock information for Apple
# apple = yf.Ticker("AAPL")

# ## Get the latest financial statement data
# financials = apple.financials 
# # Get the net income
# net_income = financials.loc['Net Income']

# # Get the latest balance sheet data
# balance_sheet = apple.balance_sheet
# # Get the total assets
# total_assets = balance_sheet.loc['Total Assets']
# total_equity = balance_sheet.loc['Total Equity Gross Minority Interest']
# #-------------------------------
# roa = net_income / total_assets
# roe = net_income / total_equity


# query the current portfolio


In [16]:
app.reqAccountUpdates(True, "DU6730183") #TODO: replace the DU222526 with our account ID
# port_event.wait()
# port_event.clear()

time.sleep(10)
last_portfolio = app.last_portfolio
app.reqAccountUpdates(False, "DU6730183") 

In [ ]:
last_portfolio

In [18]:
app.reqAccountUpdates(False, "DU6730183")#TODO: replace the DU222526 with our account ID
app.reqAccountSummary(9001, "All", "NetLiquidation")
value_event.wait()
value_event.clear()
# time.sleep(3)
total_value = app.value
app.cancelAccountSummary(9001)

ERROR:ibapi.wrapper:ERROR -1 2100 API client has been unsubscribed from account data.


Total value of the account:  1000210.66
end of account summary


In [20]:
total_value

1000210.66

# implementation
* decide what to buy & sell, as well as the quantity

In [21]:

# get the market price
for i, t in enumerate(portfolio_list):
    contract = Contract()
    contract.symbol = t
    contract.secType = "STK"
    contract.currency = "USD"
    contract.exchange = "SMART"
    app.reqMktData(i, contract, "", False, False, [])
    mkt_event.wait()
    mkt_event.clear()
    app.cancelMktData(i)

mkt_df = pd.DataFrame(app.mkt_price, columns=['reqId', 'mkt_price'])
assert sum(mkt_df.isnull().any()) == 0, "didn't get all the market price"
assert len(mkt_df) == len(portfolio_list), "didn't get all the market price"


Tick Price. Ticker Id: 0 tickType: 14 Price: 5419.0
Tick Price. Ticker Id: 0 tickType: 6 Price: 5500.0
Tick Price. Ticker Id: 0 tickType: 7 Price: 5330.51
Tick Price. Ticker Id: 0 tickType: 9 Price: 5357.92
Tick Price. Ticker Id: 0 tickType: 4 Price: 5459.78
Tick Price. Ticker Id: 1 tickType: 4 Price: 2469.19
Tick Price. Ticker Id: 1 tickType: 6 Price: 2484.79
Tick Price. Ticker Id: 1 tickType: 7 Price: 2443.7
Tick Price. Ticker Id: 1 tickType: 9 Price: 2445.74
Tick Price. Ticker Id: 1 tickType: 14 Price: 2464.08
Tick Price. Ticker Id: 1 tickType: 1 Price: 2469.47
Tick Price. Ticker Id: 1 tickType: 2 Price: 2472.28
Tick Price. Ticker Id: 2 tickType: 14 Price: 1669.56
Tick Price. Ticker Id: 2 tickType: 6 Price: 1693.97
Tick Price. Ticker Id: 2 tickType: 7 Price: 1666.03
Tick Price. Ticker Id: 2 tickType: 9 Price: 1652.56
Tick Price. Ticker Id: 2 tickType: 4 Price: 1693.0
Tick Price. Ticker Id: 2 tickType: 1 Price: 1692.63
Tick Price. Ticker Id: 2 tickType: 2 Price: 1694.71
Tick Price. T

Tick Price. Ticker Id: 9 tickType: 1 Price: 381.92
Tick Price. Ticker Id: 9 tickType: 2 Price: 383.13


In [22]:
target_portfolio = pd.DataFrame(columns=['tickers', 'weight', 'mkt_price'])
target_portfolio['tickers'] = portfolio_list
target_portfolio['weight'] = np.repeat(1/len(portfolio_list), len(portfolio_list))
target_portfolio['mkt_price'] = mkt_df['mkt_price']

In [26]:
target_portfolio['quantity'] = np.floor(target_portfolio['weight'] * (total_value * 0.9) / target_portfolio['mkt_price'])

In [27]:
last_portfolio

,ticker,quantity,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL


In [28]:
target_portfolio

,tickers,weight,mkt_price,quantity
0,NVR,0.1,5357.92,18.0
1,BKNG,0.1,2445.74,40.0
2,CMG,0.1,1652.56,60.0
3,AZO,0.1,2495.68,40.0
4,TSLA,0.1,181.41,550.0
5,URI,0.1,454.90,219.0
6,SIVB,0.1,313.38,318.0
7,REGN,0.1,758.00,131.0
8,NOW,0.1,473.81,210.0
9,MKTX,0.1,377.00,264.0


# figure out the best order of our order. (To save commission fee)

* target_portfolio
> target_portfolio python pandas dataframe containing information of stocks portfolio(target portfolio) that I want to form through trading. The first column of the dataframe is stocks' tickers and the second column of the dataframe is the quantity of each stock that I want to hold. 

* last_portfolio
> last_portfolio is a dataframe containing information of last portfolio that I hold

In [29]:
# Create a list to store the stocks that need to be sold
stocks_to_sell = []
# Create a list to store the stocks that need to buy
stocks_to_buy = []

# Iterate through the target_portfolio dataframe
for index, row in target_portfolio.iterrows():
    # Check if the stock is present in the last_portfolio dataframe
    last_portfolio_stock = last_portfolio[last_portfolio['ticker'] == row['tickers']]
    if not last_portfolio_stock.empty:
        # Compare the quantities
        last_portfolio_quantity = last_portfolio_stock['quantity'].values[0]
        if row['quantity'] < last_portfolio_quantity:
            # Calculate the difference in quantity
            quantity_diff = last_portfolio_quantity - row['quantity'] 
            # Add the stock to the list of stocks to sell
            stocks_to_sell.append([row['ticker'], quantity_diff])

        if row['quantity'] > last_portfolio_quantity:
            # Calculate the difference in quantity
            quantity_diff = row['quantity'] - last_portfolio_quantity 
            # Add the stock to the list of stocks to buy
            stocks_to_buy.append([row['ticker'], quantity_diff])

# we also need to sell the stocks that are in last_portfolio but not in target_portfolio
# Find the difference between the two portfolios
new_tickers_to_sell = set(last_portfolio['ticker']).difference(set(target_portfolio['tickers']))

for ticker in new_tickers_to_sell:
    # Find the quantity of the stock to sell from the last_portfolio
    quantity = last_portfolio.loc[last_portfolio['ticker'] == ticker, 'quantity'].values[0]
    # append to stocks to sell
    if quantity != 0:
        stocks_to_sell.append([ticker, quantity])

# we also need to buy the stocks that are in target_protfolio but not in last_portfolio
# Find the difference between the two portfolios
new_tickers_to_buy = set(target_portfolio['tickers']).difference(set(last_portfolio['ticker']))

for ticker in new_tickers_to_buy:
    # Find the quantity of the stock to buy from the target_portfolio
    quantity = target_portfolio.loc[target_portfolio['tickers'] == ticker, 'quantity'].values[0]
    # append to stocks to buy
    if quantity != 0:
        stocks_to_buy.append([ticker, quantity])

In [30]:
stocks_to_buy

[['NVR', 18.0],
 ['URI', 219.0],
 ['CMG', 60.0],
 ['TSLA', 550.0],
 ['REGN', 131.0],
 ['NOW', 210.0],
 ['AZO', 40.0],
 ['BKNG', 40.0],
 ['MKTX', 264.0],
 ['SIVB', 318.0]]

In [31]:
stocks_to_sell

[]

# place order

In [32]:
app.reqIds(-1) # require a new id
id_event.wait()
id_event.clear()
orderid = app.nextValidOrderId
for stock in stocks_to_sell:
    # Replace the place holder values with the appropriate values
    ticker = stock[0]
    quantity = stock[1]
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    order = Order()
    order.action = "SELL"
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.tif = "GTC" # default is day order

    orderid += 1

    app.placeOrder(orderid, contract, order)
    order_event.wait()
    order_event.clear()

count = 0
for stock in stocks_to_buy:
    # Replace the place holder values with the appropriate values
    app.reqAccountUpdates(False, "DU6730183")#TODO: replace the DU222526 with our account ID
    app.reqAccountSummary(9001, "All", "SettledCash")
    value_event.wait()
    value_event.clear()
    cash_value = app.value
    if count >= len(stocks_to_buy)/2 and cash_value <= 0:
        break # stop buying when cash is below 0
    ticker = stock[0]
    quantity = stock[1]
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    order = Order()
    order.action = "BUY"
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.tif = "GTC" # default is day order

    orderid += 1
    app.placeOrder(orderid, contract, order)
    order_event.wait()
    order_event.clear()
    count +=1

NextValidId: 1
openOrder. orderId:2, contract:755864,NVR,STK,,0,?,,SMART,,USD,NVR,NVR,False,,,,combo:, order:2,1000,1927015624: MKT BUY 18@0 GTC
orderStatus. orderId: 2, status: Submitted, filled: 0, remaining:18, avgFillPrice: 0.0, permId:1927015624, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
openOrder. orderId:2, contract:755864,NVR,STK,,0,?,,SMART,,USD,NVR,NVR,False,,,,combo:, order:2,1000,1927015624: MKT BUY 18@0 GTC
orderStatus. orderId: 2, status: Submitted, filled: 0, remaining:18, avgFillPrice: 0.0, permId:1927015624, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
openOrder. orderId:3, contract:3629755,URI,STK,,0,?,,SMART,,USD,URI,URI,False,,,,combo:, order:3,1000,1927015625: MKT BUY 219@0 GTC
orderStatus. orderId: 3, status: Submitted, filled: 0, remaining:219, avgFillPrice: 0.0, permId:1927015625, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
openOrder. orderId:4, contract:37655664,CMG,ST

openOrder. orderId:5, contract:76792991,TSLA,STK,,0,?,,SMART,,USD,TSLA,NMS,False,,,,combo:, order:5,1000,1927015627: MKT BUY 550@0 GTC
orderStatus. orderId: 5, status: Submitted, filled: 403, remaining:147, avgFillPrice: 193.115037, permId:1927015627, parentId:0, lastFillPrice: 193.11, clientId: 1000, whyHeld: , mktCapPrice:0.0
CommissionReport. ExecId: 00025b46.63dbd136.01.01, Commission: 0.5, Currency: USD, RealizedPnL: , Yield: , YieldRedemptionDate: 0
openOrder. orderId:5, contract:76792991,TSLA,STK,,0,?,,SMART,,USD,TSLA,NMS,False,,,,combo:, order:5,1000,1927015627: MKT BUY 550@0 GTC
orderStatus. orderId: 5, status: Submitted, filled: 403, remaining:147, avgFillPrice: 193.115037, permId:1927015627, parentId:0, lastFillPrice: 193.11, clientId: 1000, whyHeld: , mktCapPrice:0.0
CommissionReport. ExecId: 00025b46.63dbd137.01.01, Commission: 0.5, Currency: USD, RealizedPnL: , Yield: , YieldRedemptionDate: 0
openOrder. orderId:8, contract:4750,AZO,STK,,0,?,,SMART,,USD,AZO,AZO,False,,,,co

In [33]:
app.disconnect()